# Introduction
In this notebook we will write out what happens behind the scene, when we in pytorch write **loss.backward()**. This is an important [exercise](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b), as it will make us much more effective at building and debugging neural networks, by removing "the magic" happening behind the scenes.

# Libraries

In [37]:
%matplotlib inline
%config IPCompleter.use_jedi=False

In [38]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Data
We are using the same data again again!

### Reading the Data

In [39]:
# Reading names into a list
with open('../../data/names.txt', 'r') as f:
    names = f.readlines()
    names = [name.strip() for name in names]

### Creating Vocabulary

In [40]:
# Building the vocabulary (character to/from index)
chars = sorted(list(set(''.join(names))))
chr_to_idx = {s:i+1 for i,s in enumerate(chars)}; print(chr_to_idx)
chr_to_idx['.'] = 0
idx_to_chr = {i:s for s,i in chr_to_idx.items()}; print(idx_to_chr)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


### Building the Dataset

In [41]:
def build_dataset(words: list, block_size: int = 3, verbose: bool = False) -> tuple:
    """
    Description:
        Take a list of words and creates Xs and Ys give
        a block size. E.g. block_size=3 for emma would 
        result in:
            ... ---> e  
            ..e ---> m  
            .em ---> m  
            emm ---> a  
            mma ---> . 
        Stored in a (n, 3) tensor
    Inputs:
        words: List of words
        block_size: The context window
        verbose: Where to print the outputs
    Ouputs:
        A (n, block_size) tensor with Xs
        A (n) tensor with Ys
    """
    X, Y = [], []
    for word in words:
        if verbose:
            print(word)
        context = [0] * block_size
        for char in word + '.':
            idx = chr_to_idx[char]
            X.append(context)
            Y.append(idx)
            if verbose:
                print(''.join(idx_to_chr[i] for i in context), idx_to_chr[idx])
            context = context[1:] + [idx]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X, Y

In [42]:
# Putting names in random order
random.shuffle(names)

# Creating split indices
n1 = int(0.8 * len(names))
n2 = int(0.9 * len(names))

In [43]:
# Building training, validation and test sets
Xtr, Ytr = build_dataset(names[:n1]); print(Ytr.size())
Xval, Yval = build_dataset(names[n1:n2]); print(Yval.size())
Xtest, Ytest = build_dataset(names[n2:]); print(Ytest.size())

torch.Size([182501])
torch.Size([22862])
torch.Size([22783])


# Utility
Here are some utility functions for various purposes later.

In [44]:
def cmp(s: torch.Tensor, dt: torch.Tensor, t: torch.Tensor):
    """
    Description:
        Compares manually calculated gradients to gradients
        calculated using Pytorch. Prints exact gradients, 
        approximate gradients and the biggest difference
        between gradients.
    Inputs:
        s: index
        dt: Manual gradient
        t: Pytorch gradient
    """
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

# Network
We will be using one of the networks from the previous lesson that includes the bachnormalization layer.

In [45]:
n_embd = 10
n_hidden = 200
block_size = 3
vocab_size = len(chr_to_idx)

# Lookup table
C = torch.randn([vocab_size, n_embd])

# Weights and biases
## Layer 1
W1 = torch.randn([n_embd * block_size, n_hidden]) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn([n_hidden]) * 0.1

## Layer 2
W2 = torch.randn([n_hidden, vocab_size]) * 0.01
b2 = torch.randn([vocab_size]) * 0.1

## Batch Norm Shifting weights
bngain = torch.ones((1, n_hidden)) * 0.1 + 1
bnbias = torch.zeros((1, n_hidden)) * 0.1

# Placing parameters in list
parameters = [C, W1, W2, b2, bngain, bnbias]

# Enabling gradients
for p in parameters:
    p.requires_grad = True
print("Number of parameters: " + str(sum(p.nelement() for p in parameters)))

Number of parameters: 12097


# Training
 The training loop will also be almost the same except that **loss.backward()** will be written manually. One change is that the forward has been chunked such that we can debug loss.backward() on one step at the time.

### Constructing a mini-batch

In [46]:
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,))
Xb, Yb = Xtr[ix], Ytr[ix]

### Chunked Forward Pass

In [47]:
# Embeddings
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

# Linear layer
hprebn = embcat @ W1 + b1

# BatchNorm layer
bnmeani = 1/batch_size*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer
logits = h @ W2 + b2 # output layer

# Loss (cross entropy loss)
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(batch_size), Yb].mean()

### Pytorch Backward Pass

In [48]:
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3083, grad_fn=<NegBackward0>)

# Exercise 1
Here we will backprop through the all of the variables as they are defined in the forward pass above. 

#### Derivative of logprobs
An example of the mean operation (see last calculation in the forward pass).

Calculation:  
loss = -(a + b + c) / 3 <=>  
loss = -1/3\*a + -1/3\*b + -1/3\*c

Derivative:  
dloss/da = -1/3 + 0 + 0 <=>  
dloss/da = -1/3  

As only the predicted letters participate in the loss, the remaning letters have a gradient of zero.

In [49]:
# Initializing with zeroes
dlogprobs = torch.zeros_like(logprobs)

# Setting gradients in letter locations
dlogprobs[range(batch_size), Yb] = -1.0/batch_size

# Checking estimated gradients vs pytorch gradients
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


#### Derivative of probs
The calculation can be found in the forward pass.

Calculation:  
logprobs = log(a)  

Derivative:  
dlogprobs/da = 1/a    

Chain Rule:
F(x) = f(g(x)) =>  
F'(x) = f'(g(x)) * g'(x)  

In our case this means that:  
F'(x) = 1/probs * dlogprobs

In [50]:
# Setting gradients
dprobs = 1.0/probs * dlogprobs

# Checking estimated gradients vs pytorch gradients
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


#### Derivative of counts_sum_inv
From here on only the calculation of the derivative will be show here, while the chain rule will be applied in the actual calculations.

Calculation:  
counts_sum_inv = a * b  

Derivative:  
dcounts_sum_inv/da = b    

In [51]:
# Setting gradients
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

# Checking estimated gradients vs pytorch gradients
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


#### Calculating the remaining gradients
We will no longer go through explicit examples, but just show the calculations. How to calculate the different derivatives can be found on [Wolframalpha](https://www.wolframalpha.com/input?i=d%2Fdx+x%5E-1)

In [52]:
# counts (part 1)
dcounts = counts_sum_inv * dprobs

In [53]:
# counts_sum
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


In [54]:
# counts (part 2)
dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


In [55]:
# norm_logits
dnorm_logits = counts * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


In [56]:
# logit_maxes
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0


In [57]:
# logits
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

logits          | exact: True  | approximate: True  | maxdiff: 0.0


#### The derivative for matrix multiplication
Here we make a small example, which illustrates how to calculate the derivatives.

<pre>
d = a @ b + c

| d11 d12 |       | a11 a12 |     | b11 b12 |    | c1 c2 |
| d21 d22 |   =   | a21 a22 |  @  | b21 b22 |  + | c1 c2 | =>

d11 = a11 * b11 + a12 * b21 + c1
d12 = a11 * b12 + a12 * b22 + c2
d21 = a21 * b11 + a22 * b21 + c1
d22 = a21 * b12 + a22 * b22 + c2
</pre>

When finding the derivative with respect to a, it will be:

<pre>
dL/da11 = dL/dd11 * b11 + dL/dd12 * b12
dL/da12 = dL/dd11 * b21 + dL/dd12 * b22
dL/da21 = dL/dd21 * b11 + dL/dd22 * b12
dL/da22 = dL/dd21 * b21 + dL/dd22 * b22
</pre>

This means that the derivative for d is (if rewritten as a matrix multiplication):

<pre>

          | dL/da11    dL/da12 |   | b11    b21 |
dL/da  =  | dL/da21    dL/da22 | * | b12    b22 | <=>

dL/da = dL/dd @ b^T
</pre>

Doing the same for the derivative with respect to b, you will get

<pre>
dL/db = a^T @ dL/dd 
</pre>

Doing the same for the derivative with respect to c, you will get

<pre>
dL/dc = dL/dd @ sum(0)
</pre>

In [58]:
# h
dh = dlogits @ W2.T
cmp('h', dh, h)

h               | exact: True  | approximate: True  | maxdiff: 0.0


In [59]:
# W2
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

W2              | exact: True  | approximate: True  | maxdiff: 0.0


In [60]:
# b2
db2 = dlogits.sum(0)
cmp('b2', db2, b2)

b2              | exact: True  | approximate: True  | maxdiff: 0.0


In [61]:
# hpreact
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

hpreact         | exact: False | approximate: True  | maxdiff: 5.820766091346741e-11


In [62]:
# bngain
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

bngain          | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-10


In [63]:
# bnraw
dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

bnraw           | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-10


In [64]:
# bnbias
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

bnbias          | exact: False | approximate: True  | maxdiff: 3.4924596548080444e-10


In [69]:
# bndiff (part 1)
dbndiff = bnvar_inv * dbnraw

In [70]:
# bnvar_inv 
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

bnvar_inv       | exact: False | approximate: True  | maxdiff: 2.9103830456733704e-10


In [71]:
# bnvar
dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

bnvar           | exact: False | approximate: True  | maxdiff: 5.820766091346741e-11


In [72]:
# bndiff2
dbndiff2 = (1.0/(batch_size-1))*torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

bndiff2         | exact: False | approximate: True  | maxdiff: 1.8189894035458565e-12


In [73]:
# bndiff (part 2)
dbndiff += (2*bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

bndiff          | exact: False | approximate: True  | maxdiff: 5.820766091346741e-11


In [74]:
# CONTINUE: http://www.youtube.com/watch?v=q8SA3rM6ckI&t=66m0s

In [ ]:
# hprebn (part 1)
dhprebn = dbndiff.clone()

In [ ]:
# bnmeani
dbnmeani = (-dbndiff).sum(0)
cmp('bnmeani', dbnmeani, bnmeani)

In [ ]:
# hprebn (part 2)
dhprebn += 1.0/batch_size * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)

In [ ]:
# embcat
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

In [ ]:
# W1
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

In [ ]:
# b1
db1 = dhprebn.sum(0)
cmp('b1', db1, b1)

In [ ]:
# emb
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

In [ ]:
# C
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]
cmp('C', dC, C)

# Exercise 2

# Exercise 3

# Exercise 4